In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider
import scipy

# GX 01
You are given the complex field `01_unknown.npy`. Load the complex array with `np.load("01_unknown.npy")`.
The wavelength is 633nm and the physical pixel size of the image is 1µm.


## Tasks
### 1
What is the correct negative propagation (backwards) distance to obtain a sharp, meaningful image?


### 2 Bonus
What happens if you scale the pixel size and the propagation distance by a factor of 10. Does it still work? Try it out and explain what happens.
What would you need to change to make it work again.